In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
import tensorflow as tf
import numpy as np
import zipfile
import os
import shutil
tf.keras.backend.clear_session()



In [2]:
train_cats_dir = 'catsvdogs/train/cats'
train_dogs_dir = 'catsvdogs/train/dogs'
print("No of cat images: ",len(os.listdir(train_cats_dir)))
print("No of dog images: ",len(os.listdir(train_dogs_dir)))

No of cat images:  12500
No of dog images:  12500


In [3]:
da = 0.9

class cb(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = {}):
    if logs.get('val_acc') is not None and logs['val_acc'] > da:
      print("\nTraining stopped as validation accuracy crossed the threshold.")
      self.model.stop_training = True

cback = cb()

In [4]:
model = tf.keras.Sequential([
                             tf.keras.layers.Conv2D(16, kernel_size = (3,3), activation = tf.nn.relu, input_shape = (150,150,3)),
                             tf.keras.layers.MaxPool2D((2,2)),
                             tf.keras.layers.Conv2D(32, kernel_size = (3,3), activation = tf.nn.relu),
                             tf.keras.layers.MaxPool2D((2,2)),
                             tf.keras.layers.Conv2D(64, kernel_size = (3,3), activation = tf.nn.relu),
                             tf.keras.layers.MaxPool2D((2,2)),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(512, activation = tf.nn.relu),
                             tf.keras.layers.Dense(1, activation = tf.nn.sigmoid)
])

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = 'accuracy')

In [5]:
direc = 'catsvdogs/train/'

training_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255, validation_split = 0.1, 
width_shift_range = 0.2, height_shift_range = 0.2, zoom_range = 0.2, shear_range = 0.2, rotation_range = 40, fill_mode = 'nearest', horizontal_flip = True)
training_gen = training_datagen.flow_from_directory(
    direc,
    target_size = (150, 150),
    class_mode = 'binary',
    batch_size = 50,
    subset = ('training')
)


validation_gen = training_datagen.flow_from_directory(
    direc,
    target_size = (150, 150),
    class_mode = 'binary',
    batch_size = 50,
    subset = ('validation')
)

Found 22500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [6]:
model.fit(training_gen, steps_per_epoch = 450, epochs = 5, validation_data = validation_gen, validation_steps = 50)

Epoch 1/5
 48/450 [==>...........................] - ETA: 31s - loss: 0.7270 - accuracy: 0.5404

KeyboardInterrupt: 

In [7]:
tf.keras.backend.clear_session()